In [1]:
import sys
import torch

sys.path.append('../')

/home/tsa87/anaconda3/envs/tacogfn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

from src.tacogfn.envs import frag_mol_env
from src.tacogfn.models import pharmaco_cond_graph_transformer
from src.tacogfn.data.utils import merge_pharmacophore_and_molecule_data_list
import utils

In [3]:
ctx = frag_mol_env.FragMolBuildingEnvContext()

pharmacophore_data_list = utils.get_example_pharmacophore_datalist()
molecule_data_list = utils.get_example_molecule_datalist(ctx)
batch = merge_pharmacophore_and_molecule_data_list(pharmacophore_data_list, molecule_data_list)

In [ ]:
ctx.num_edge_dim

14

In [ ]:
model = pharmaco_cond_graph_transformer.PharmacophoreConditionalGraphTransformer(
    pharmacophore_dim=64,
    x_dim=ctx.num_node_dim,
    e_dim=ctx.num_edge_dim,
    g_dim=ctx.num_cond_dim,
    num_emb=64,
    num_layers=3,
    num_heads=2,
    ln_type="pre",
)

In [ ]:
model(batch, torch.randn(len(batch), ctx.num_cond_dim))[1].shape

torch.Size([10, 128])